In [1]:
import subprocess
import re
import pandas as pd

In [2]:
def run_GA(n_iterations: int, 
           n_individuals: int, 
           genome_length: int, 
           probability_crossover: float, 
           probability_mutation: float):
    executable = "./cmake-build-release/main"
    result = subprocess.run([executable, str(n_iterations), str(n_individuals), str(genome_length), str(probability_crossover), str(probability_mutation)], stdout=subprocess.PIPE)
    
    #print(f"Error code: {result.returncode}\n\n")
    
    results = pd.DataFrame(columns=['Best_idx','Best_fitness','Worst_idx','Worst_fitness','Avg_fitness','Best_genome'])
    regex_float = "[+-]?([0-9]*[.])?[0-9]+"
    regex_int = "[0-9]*"
    for line in result.stdout.decode("utf-8").split("\n"):
        match = re.search(f'Iter ({regex_int}): Best: ({regex_int})->({regex_float})  \|\| Worst: ({regex_int})->({regex_float})  \|\| Avg_fitness: ({regex_float})  \|\| Best_genome: ([\d ]*)', line)
        if match:
            results.loc[len(results.index)] = [match.group(2), match.group(3), match.group(5), match.group(6), match.group(8), match.group(10)]
    return results

In [29]:
n_iterations = 1000
repetitions = 30
runs = {}
for individuals in range(25,55,5):
    for genome_length in [2,3,4,8,10]:
        for prob_crossover in range(0,12,2):
            for prob_mutation in range(0,12,2):
                id_run = f"{individuals} {genome_length} {prob_crossover/10} {prob_mutation/10}"
                runs[id_run] = {"i":individuals, "gl":genome_length, "pc":prob_crossover, "pm":prob_mutation}
#print(runs)
#result = run_GA(100, 2, 2, 1, 1)

In [32]:
from multiprocessing import Pool
import tqdm

with tqdm.tqdm(total=len(runs)) as pbar:
    def f(x):
        row = runs[x]
        result = []
        for i in range(repetitions):
            result.append(run_GA(100, row["i"], row["gl"], row["pc"], row["pm"]))
        pbar.update()
        return x, result
    with Pool(8) as p:
        results_run = p.map(f, runs)

  0%|                                                                                                                                                                                         | 0/1080 [01:01<?, ?it/s]


KeyboardInterrupt: 

In [25]:
print(results_run)

[('5 2 0.0 0.0',    Best_idx Best_fitness Worst_idx Worst_fitness Avg_fitness Best_genome
0         0      743.794         4      1133.626     878.301       12 4 
1         0      743.794         1       912.791     811.371       12 4 
2         0      743.794         4       912.791     811.371       12 4 
3         0      743.794         1       802.321     789.277       12 4 
4         0      743.794         3       802.321     788.608       12 4 
..      ...          ...       ...           ...         ...         ...
95        0      743.794         1       798.975     787.939       12 4 
96        0      743.794         1       798.975     787.939       12 4 
97        0      743.794         1       798.975     787.939       12 4 
98        0      743.794         1       798.975     787.939       12 4 
99        0      743.794         1       798.975     787.939       12 4 

[100 rows x 6 columns]), ('5 4 0.0 0.0',    Best_idx Best_fitness Worst_idx Worst_fitness Avg_fitness  Bes